**This script keeps track of a signrequest status for particular documents designated by template ID**

**Add local library to path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [2]:
import pandas as pd
import numpy as np
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import signrequest_documents
from local_library import upload_to_gdrive


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Download Documents Functions Library**

In [3]:
def import_download_df():
    """
    Imports the student and signrequests dfs and merges them
    
    Args:
        N/A
        
    Returns:
        download_df: A dataframe containing merged signrequests and students data.
    """
    
    #import signrequest update sheet
    signrequests_df = import_worksheet("Tech Mentors 2021", "Status Updates")

    #import student contract data
    student_df = import_worksheet("Tech Mentors 2021", "Student Data")
    
    #create dataframe columns
    student_columns = ["First name", "Campus","Surname", "Email"]
    signrequests_columns = ["email", "mentor document", "mentor contract status","mentor document url"]
    download_columns = ["First name", "Surname", "Email", "mentor document", "mentor contract status", "mentor document url"]

    #subset dataframes using columns
    student_df = student_df[student_columns]
    signrequests_df = signrequests_df[signrequests_columns]

    #merge signrequests with students dataframes
    download_df = pd.merge(student_df, signrequests_df, how="left", left_on="Email", right_on="email")
    
    return download_df[download_columns]


def create_folder_ids(download_df):
    """
    Creates folder ids which represent the folder where the document will be saved.
    
    Args:
        target_df: A dataframe which will contain the folder ids.
        
    Returns:
        _: None
    """
    
        
    #map to folder id's
    download_folder_map = {"DBN Mentor Contract":"1R2OKfgQQBbHJHyOTZEwxnaEmfVHUbKEj", "CPT Mentor Contract":"135N7dMe099Xmo3NOCdzEGchgfkbjnfnv",
                         "JHB Mentor Contract":"1rfGuq3hJZOmqhSzo57OO9Ar0gI0XICwk"}
    
    #create contract file ids
    download_df["mentor_folder_id"] = download_df["mentor document"].map(download_folder_map)
    
    pass


def create_file_names(download_df):
    """
    Creates file names for the files to be saved.
    
    Args:
        download_df: A dataframe containing the data to create the filenames.
        
    Returns:
        _: None
    """
    
    #create contract file names
    download_df["contract_file_name"] = download_df["First name"].str.strip() + " " + download_df["Surname"].str.strip() + " " + "Mentor-Contract"

    pass


def create_status_columns(download_df):
    """
    Creates status updates columns for the input dataframe.
    
    Args:
        download_df: A dataframe for which the status updates columns will be created.
        
    Returns:
        _: None
    """
    
    #create contract status column
    download_df["mentor contract status"] = np.nan

    pass


def create_url_columns(download_df):
    """
    Creates status updates columns for the input dataframe.
    
    Args:
        download_df: A dataframe for which the status updates columns will be created.
        
    Returns:
        _: None
    """
    
    #create contract status column
    download_df["mentor document url"] = np.nan

    pass

    
def subset_download_df(download_df):
    """
    Subsets the download dataframe using set out columns.
    
    Args:
        download_df: A dataframe for which the status updates columns will be created.
        
    Returns:
        _: None
    """
    
    #create download columns
    download_columns = ["First name", "Surname", "Email", "mentor_file_name", "mentor contract status", "mentor_folder_id"]

    #subset download dataframe using download columns
    return download_df[download_columns]
    

def create_download_df():
    """
    Creates the dataframe containing all the information regarding dowloading files.
    
    Args:
        N/A
        
    Returns:
        download_df: A dataframe containing all the information regarding dowloading files.
    """
    download_df = import_download_df()
    create_folder_ids(download_df)
    create_file_names(download_df)
    create_status_columns(download_df)
    download_df = subset_download_df(download_df)

    return download_df
    
    
def get_download_df():
    """
    Gets or creates the dataframe containing all data for file downloads.
    
    Args:
        N/A
        
    Returns:
        download_df: A dataframe containing all the information regarding dowloading files.
    """

    #import download update sheet
    download_df = import_worksheet("Tech Mentors 2021", "Downloaded Documents")
    if download_df.empty:
        download_df = create_download_df()
        export_worksheet("Tech Mentors 2021", "Downloaded Documents", download_df)
    create_url_columns(download_df)
    return download_df
    
    



def bulk_upload_to_gdrive(download_df, target_df, upload_map):
    """
    Bulk uploads contracts to google drive.
    
    Args:
    download_df: A dataframe containing all the links to be downloaded and uploaded.
    upload_map: A dictionary containing the column names to be used
    
    Returns:
        N/A
    """
    
    for index, row in target_df.iterrows():
        
        if upload_to_gdrive(row[upload_map["document_url"]], row[upload_map["file_name"]], row[upload_map["folder_id"]]):
            download_df.loc[download_df[upload_map["file_name"]] == row[upload_map["file_name"]], upload_map["document_status"]] = 'Saved'
        else:
            print(f"The following file could not be uploaded: {row[upload_map['file_name']]}")
    return


**Local Functions Library**

In [4]:




def relevant_keys(target_columns):
    """
    Checks that all relevant keys are present.
    
    Args:
        target_columns: All columns of a dataframe.
        
    Returns:
        _: A boolean that indicates that all keys are relevant or not.
    """
    
    if 'template_id' not in target_columns:
        return False
    if 'email' not in target_columns:
        return False
    if 'signrequest_status' not in target_columns:
        return False
    if 'document_url' not in target_columns:
        return False
    return True


def add_download_links(download_df, links_df):
    """
    Adds download links to the download df dataframe.
    
    Args:
        download_df: A dataframe containing all the information regarding dowloading files.
        links_df: A dataframe containing links to the documents.
        
    Returns:
        updated_download_df: A dataframe resulting in merging download_df with links_df.
    """
    
    #filter links
    links_df = links_df[links_df['signrequest_status'].isin(["signed", "downloaded", "signed and downloaded"])]
    links_df = links_df.dropna()
    
    #update download_df
    for index, row in links_df.iterrows():
        condition_email = download_df['Email'] == row['email']
        condition_contract = download_df['mentor_file_name'].str.contains(row['template_id'], case=False)
        if download_df[condition_email & condition_contract].shape[0] == 1:
            download_df.loc[condition_email & condition_contract, 'mentor document url'] = row['document_url']
    pass


def upload_documents(download_df, links_df):
    """
    Uploads documents to google drive
    
    Args:
        download_df: A dataframe containing all the document data.
        links_df: A dataframe containing links to the documents.
        
    Returns:
        N/A
    """
    
    campus_document_map = {"a95c9077-ccdb-48f2-b55f-4e2b5d7f35b9":"contract", "c28bcf73-98e9-498d-94a2-26f9b7b540b9":"conduct",
                      "280a79c3-d773-466a-8db1-e2a5c5394b60":"contract", "08c38bcc-0080-40f4-b0b6-4276712887a2":"conduct",
                      "daf83fcc-c68b-4ef0-9ea7-18420732d818":"contract", "fc1e9cfc-fbd2-4fc6-b852-75d658819670":"conduct"}
    
    #check all columns are present
    if not relevant_keys(links_df.columns):
        return
    
    links_df['template_id'] = links_df['template_id'].map(campus_document_map)
    add_download_links(download_df, links_df)
    
    #set vars for contract
    contract_map = {"document_url":"contract document url", "file_name":"contract_file_name",
                    "document_status":"contract status", "folder_id":"contract_folder_id"}
    condition_not_null = download_df['contract document url'].notnull()
    condition_not_saved = download_df['contract status'] != "Saved"
    contract_df = download_df[condition_not_null & condition_not_saved]
    bulk_upload_to_gdrive(download_df, contract_df, contract_map)
    
    #set vars for code
    code_map = {"document_url":"code of conduct document url", "file_name":"code_of_conduct_file_name",
                    "document_status":"code of conduct status", "folder_id":"code_of_conduct_folder_id"}
    condition_not_null = download_df['code of conduct document url'].notnull()
    condition_not_saved = download_df['code of conduct status'] != "Saved"
    conduct_df = download_df[condition_not_null & condition_not_saved]
    bulk_upload_to_gdrive(download_df, conduct_df, code_map)
    
    return

def get_documents_metadata(source_df, download_df):
    """
    Gets API metadata for all signrequests for a particular template.
    
    Args:
        source_df: A dataframe
        download_df: A dataframe.
        
    Returns:
        signrequests_list: A list of all relevant SignRequests.
    """
    
    #set variables
    signrequests_list = []
    page_number = 1
    
    #get unique template ids and emails
    emails = get_unique_values(source_df, "Email Address")
    template_ids = get_unique_values(source_df, ["Student Contract ID", "Code of Conduct ID"])
    
    #get pages
    while True:
        specific_filtered_results, next_page = signrequest_documents(emails, template_ids, page_number)
        upload_documents(download_df, pd.DataFrame(specific_filtered_results))
        signrequests_list += specific_filtered_results
        if next_page:
            page_number += 1
        else:
            break

    return signrequests_list


def get_spreadsheet_data():
    """
    Gets and refines the signrequest source data.
    
    Args:
        N/A
        
    Returns:
        source_df: A dataframe containing refined source data used for the signrequests.
    """
    
    #import student contract data
    student_df = import_worksheet("SRS Cohort 2021 Contracting", "Applicants")

    #import contract templates id
    template_df = import_worksheet("SRS Cohort 2021 Contracting", "Template IDs")
    
    #merge student data and template data
    source_df = pd.merge(student_df, template_df, on="Campus")
    
    #list relevant columns
    source_df_columns = ["Email Address", "Student Contract ID", "Code of Conduct ID"]

    #subset columns based on list of relevant columns
    source_df = source_df[source_df_columns]
    
    return source_df


def get_unique_values(target_df, columns):
    """
    Gets all the unique values in a(the) specified column(s).
    
    Args:
        target_df: A datframe containing the values in one or more columns.
        columns: A string or list of strings representing value containing columns.
        
    Returns:
        _: A list of unique values.
    """
    
    target_columns = list(filter(lambda column: True if column in columns else False ,target_df.columns))
    return pd.unique(target_df[target_columns].values.ravel('K'))


def add_signrequest_status_updates(target_df, results_list):
    """
    Adds status updates to the target df based on template id values.
    
    Args:
        target_df: The dataframe to which updates are added.
        template_columns: The columns containing the template ids.
        updates: A list of objects containing signrequests data.
        
    Returns:
        N/A
    """

    results_df = pd.DataFrame(results_list)
    contracts = ["a95c9077-ccdb-48f2-b55f-4e2b5d7f35b9", "280a79c3-d773-466a-8db1-e2a5c5394b60", "daf83fcc-c68b-4ef0-9ea7-18420732d818"]
    code_of_conduct = ["c28bcf73-98e9-498d-94a2-26f9b7b540b9", "08c38bcc-0080-40f4-b0b6-4276712887a2", "fc1e9cfc-fbd2-4fc6-b852-75d658819670"]
    results_df["contracts"] = [True if template_id in contracts else False for template_id in results_df["template_id"]]
    results_df["code_of_conduct"] = [True if template_id in code_of_conduct else False for template_id in results_df["template_id"]]
    contracts_df = results_df[results_df["contracts"]]
    code_of_conduct_df = results_df[results_df["code_of_conduct"]]
    results_columns = ["email", "template_id", "signrequest_status", "document_url"]
    contracts_columns = ["email", "template_id", "contracts status", "contracts document_url"]
    code_of_conduct_columns = ["email", "template_id", "code of conduct status", "code of conduct document_url"]
    contracts_df = contracts_df[results_columns]
    contracts_df.columns = contracts_columns
    code_of_conduct_df = code_of_conduct_df[results_columns]
    code_of_conduct_df.columns = code_of_conduct_columns
    new_results_df = pd.merge(contracts_df, code_of_conduct_df, on=["email"], how="left")
    campus_document_map = {"a95c9077-ccdb-48f2-b55f-4e2b5d7f35b9":"CPT Contract", "c28bcf73-98e9-498d-94a2-26f9b7b540b9":"CPT Code of Conduct",
                      "280a79c3-d773-466a-8db1-e2a5c5394b60":"Durban Contract", "08c38bcc-0080-40f4-b0b6-4276712887a2":"Durban Code of Conduct",
                      "daf83fcc-c68b-4ef0-9ea7-18420732d818":"JHB Contract", "fc1e9cfc-fbd2-4fc6-b852-75d658819670":"JHB Code of Conduct"}
    new_results_df["template_id_x"] = new_results_df["template_id_x"].map(campus_document_map)
    new_results_df["template_id_y"] = new_results_df["template_id_y"].map(campus_document_map)
    new_columns = ["email", "contract document", "contract status", "code of conduct document", "code of conduct status", "contract document url" ,"code of conduct document url"]
    old_columns = ["email", "contract document", "contract status", "contract document url", "code of conduct document", "code of conduct status" ,"code of conduct document url"]
    new_results_df.columns = old_columns
    
    return new_results_df[new_columns]





**Execute Program**

In [5]:
#get signrequests source data
source_df = get_spreadsheet_data()

#get download dataframe
download_df = get_download_df()

#get signrequests metadata
results_list = get_documents_metadata(source_df, download_df)

#add signrequest status updates to source dataframe
updated_df = add_signrequest_status_updates(source_df, results_list)

In [7]:
#export updated signrequest source dataframe to google worksheet
#export_worksheet("SRS Cohort 2021 Contracting" ,"Status Updates", updated_df)
#export_worksheet("SRS Cohort 2021 Contracting", "Downloaded Documents", download_df)

,email,contract document,contract status,code of conduct document,code of conduct status,contract document url,code of conduct document url
0,hlngoben020@student.wethinkcode.co.za,JHB Contract,signed,JHB Code of Conduct,signed,https://signrequest-pro.s3.amazonaws.com/pdfs/...,https://signrequest-pro.s3.amazonaws.com/pdfs/...
1,lchauke020@student.wethinkcode.co.za,JHB Contract,signed,JHB Code of Conduct,signed,https://signrequest-pro.s3.amazonaws.com/pdfs/...,https://signrequest-pro.s3.amazonaws.com/pdfs/...
2,mmabanti020@student.wethinkcode.co.za,JHB Contract,signed,JHB Code of Conduct,signed,https://signrequest-pro.s3.amazonaws.com/pdfs/...,https://signrequest-pro.s3.amazonaws.com/pdfs/...
3,gdindi020@student.wethinkcode.co.za,JHB Contract,signed,JHB Code of Conduct,signed,https://signrequest-pro.s3.amazonaws.com/pdfs/...,https://signrequest-pro.s3.amazonaws.com/pdfs/...
4,tlethoko020@student.wethinkcode.co.za,JHB Contract,signed,JHB Code of Conduct,signed,https://signrequest-pro.s3.amazonaws.com/pdfs/...,https://signrequest-pro.s3.amazonaws.com/pdfs/...
